In [ ]:
import sys
from pathlib import Path
# Add tools directory so Matcher can be imported (notebook is in tools/matching_related/)
_cwd = Path.cwd()
if (_cwd / "Matcher").exists():
    _tools = _cwd
elif (_cwd.parent / "Matcher").exists():
    _tools = _cwd.parent
elif (_cwd / "tools" / "Matcher").exists():
    _tools = _cwd / "tools"
else:
    _tools = _cwd.parent
if str(_tools) not in sys.path:
    sys.path.insert(0, str(_tools))

In [ ]:
%reload_ext autoreload
%autoreload 2

from Matcher.utilities import DataLoader
from Matcher.scorer import ScorerConfig, PreferenceScorer, SimilarityScorer
from Matcher.matcher import Matcher
import numpy as np

In [ ]:
dataloader = DataLoader()
config = ScorerConfig(

)

heterosexual_female_df = dataloader.load_data("embedded_heterosexual_female_df")
heterosexual_male_df = dataloader.load_data("embedded_heterosexual_male_df")
homosexual_female_df = dataloader.load_data("embedded_homosexual_female_df")
homosexual_male_df = dataloader.load_data("embedded_homosexual_male_df")

In [ ]:
FM_preference_scorer = PreferenceScorer(config, heterosexual_female_df, heterosexual_male_df)
MF_preference_scorer = PreferenceScorer(config, heterosexual_male_df, heterosexual_female_df)
MM_preference_scorer = PreferenceScorer(config, homosexual_male_df, homosexual_male_df)
FF_preference_scorer = PreferenceScorer(config, homosexual_female_df, homosexual_female_df)

FM_preference_res = FM_preference_scorer.calculate_score_matrix()
MF_preference_res = MF_preference_scorer.calculate_score_matrix()
MM_preference_res = MM_preference_scorer.calculate_score_matrix()
FF_preference_res = FF_preference_scorer.calculate_score_matrix()


In [ ]:
FM_similarity_scorer = SimilarityScorer(config, heterosexual_female_df, heterosexual_male_df)
MF_similarity_scorer = SimilarityScorer(config, heterosexual_male_df, heterosexual_female_df)
MM_similarity_scorer = SimilarityScorer(config, homosexual_male_df, homosexual_male_df)
FF_similarity_scorer = SimilarityScorer(config, homosexual_female_df, homosexual_female_df)

FM_similarity_res = FM_similarity_scorer.calculate_score_matrix()
MF_similarity_res = MF_similarity_scorer.calculate_score_matrix()
MM_similarity_res = MM_similarity_scorer.calculate_score_matrix()
FF_similarity_res = FF_similarity_scorer.calculate_score_matrix()

In [ ]:
total_FM = FM_preference_res + FM_similarity_res
total_MF = MF_preference_res + MF_similarity_res
total_MM = MM_preference_res + MM_similarity_res
total_FF = FF_preference_res + FF_similarity_res

total_MM[np.arange(len(total_MM)), np.arange(len(total_MM))] = -np.inf
total_FF[np.arange(len(total_FF)), np.arange(len(total_FF))] = -np.inf
total_FM = np.min([total_FM, total_MF.T], axis=0)

clipped_FM = np.clip(total_FM, min=0)
clipped_MM = np.clip(total_MM, min=0)
clipped_FF = np.clip(total_FF, min=0)


In [ ]:
hetrosexual_matcher = Matcher(clipped_FM)
hetrosexual_matching_result = hetrosexual_matcher.hungarian()

In [ ]:
len_f, len_m = total_FM.shape
for f_idx, m_idx in hetrosexual_matching_result:
    if f_idx < len_f and m_idx < len_m:
        print(f"F{f_idx} <-> M{m_idx}")
    elif f_idx >= len_f:
        print(f"M{m_idx} did not get a match")
    elif m_idx >= len_m:
        print(f"F{f_idx} did not get a match")

In [ ]:
MM_matcher = Matcher(clipped_MM)
MM_matching_result = MM_matcher.hungarian()
len_m, len_m = total_MM.shape
for m1_idx, m2_idx in MM_matching_result:
    if m1_idx < len_m and m2_idx < len_m:
        print(f"M{m1_idx} -> M{m2_idx}")

In [ ]:
FF_matcher = Matcher(clipped_FF)
FF_matching_result = FF_matcher.hungarian()
len_f, len_f = total_FF.shape
for f1_idx, f2_idx in FF_matching_result:
    if f1_idx < len_f and f2_idx < len_f:
        print(f"F{f1_idx} -> F{f2_idx}")
